<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-EarthPurse/blob/main/pivot_point_dirven_pendulum_workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Setting up the pivot simulation
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Constants for gravity and lenght of pendulum
g = 9.81      # gravitational acceleration (m/s^2)
l = 1.0       # pendulum length (m)
omega_d = 5.0 # drive frequency (rad/s)

# Pivot motion
def get_pivot_xy(t, omega_d, l):
    x_p = -l * np.sin(omega_d * t)
    y_p = l * (np.cos(omega_d * t) - 1)
    return x_p, y_p

# Equation for motion
def pendulum_equations(t, y, l, g, omega_d):
    theta, theta_dot = y
    # second derivatives of pivot position
    xpp = l * (omega_d**2) * np.sin(omega_d * t)
    ypp = -l * (omega_d**2) * np.cos(omega_d * t)
    # angular acceleration
    theta_ddot = (-g * np.sin(theta) - xpp * np.cos(theta) - ypp * np.sin(theta)) / l
    return [theta_dot, theta_ddot]

# Solving the equation
t_min, t_max = 0, 10
t_eval = np.linspace(t_min, t_max, 1000)
y0 = [0.0, 0.0]  # initial conditions: theta=0, theta_dot=0
sol = solve_ivp(pendulum_equations, [t_min, t_max], y0, t_eval=t_eval, args=(l, g, omega_d), dense_output=True)

# Converting coordinates
def get_pendulum_xy(t, theta, l):
    x = l * np.sin(theta)
    y = -l * np.cos(theta)
    return x, y
def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum):
    return x_pivot + x_pendulum, y_pivot + y_pendulum

# Animated function
def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    fig, ax = plt.subplots(figsize=(4, 4))
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory')

    def animate(i):
        pivot.set_data([x_pivot[i]], [y_pivot[i]])
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]])

        artists = [pivot, pendulum_arm, pendulum_bob]
        if show_traj and trajectory:
            trajectory.set_data(x_lab[:i+1], y_lab[:i+1])
            artists.append(trajectory)
        return artists

    average_time_step = np.mean(np.diff(t))
    interval = average_time_step * 1000 * speed_factor

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig)
    return anim

# Generated Animation
n_points = 300
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0]

x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, l)
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum)

pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, l=l, show_traj=True, speed_factor=5)
HTML(pendulum_animation.to_html5_video())

# Analytical stability analysis
"""
Stability Analysis:
-------------------
For small oscillations (theta ≈ 0), the pivot's circular motion introduces an effective oscillatory forcing term.
- At low driving frequencies, the pivot motion disturbs the pendulum, making θ = 0 unstable.
- At high frequencies, averaging over rapid motion creates an effective potential minimum at θ = 0.
"""



"\nStability Analysis:\n-------------------\nFor small oscillations (theta ≈ 0), the pivot's circular motion introduces an effective oscillatory forcing term.\n- At low driving frequencies, the pivot motion disturbs the pendulum, making θ = 0 unstable.\n- At high frequencies, averaging over rapid motion creates an effective potential minimum at θ = 0.\n"